<a href="https://colab.research.google.com/github/prateekkosta/Retail-Data-cleaning-and-EDA/blob/main/Copy_of_Retail_Data_Cleaning_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Objective**- Perform EDA to find out how many customers are happy with the product and service provided by the retail company.

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

**Reading Data**

In [ ]:
dirty_data= pd.read_csv('dirty_data.csv')
dirty_data.head()

Get The Description of Data

In [ ]:
dirty_data.describe()

In [ ]:
# Shape of the Data
print('Shape of the Dirty data=', dirty_data.shape)

Checking Null values

In [ ]:
# check for null values
print(dirty_data.isnull().sum())

**Box Plot for Checking Ouliers with missing values**

In [ ]:
plt.figure(figsize=(8,8))
dirty_data.boxplot(column=['delivery_charges', 'order_price'])
plt.title('Box plot on Columns with Null values')
plt.xlabel('Columns')
plt.ylabel('Values')
plt.show()

**Replacing of Null Values with Mean, Median and Mode**

In [ ]:
# replacing null values of Delivery Charges with mean

dirty_data['delivery_charges'].fillna(dirty_data['delivery_charges'].mean(), inplace= True)

In [ ]:
# replacing null values of Order Price with Median because it has multiple outliers

dirty_data['order_price'].fillna(dirty_data['order_price'].median(), inplace= True)

In [ ]:
# replacing null values of Is Happy Customer column with Mode because binary balue columns are replaced with mode

dirty_data['is_happy_customer'].fillna(dirty_data['is_happy_customer'].mode()[0], inplace= True)

In [ ]:
dirty_data['is_happy_customer'].mode()

In [ ]:
# again checking for null values

print(dirty_data.isnull().sum())

In [ ]:
# checking the data type

print(dirty_data.dtypes)



Conversion of Datatype

In [ ]:

# Converting Date column into Date time format

dirty_data.date= pd.to_datetime(dirty_data['date'])

In [ ]:
print(dirty_data['date'].dtypes)

**Sanity Check Of Data**

In [ ]:
# Cheking for Unique Data in Order ID column

if dirty_data.order_id.is_unique:
  print('Data in Order Id is Unique.')
else:
  print('Data in Order Id in not Unique.')

In [ ]:
# checking for unique values in Column

dirty_data['nearest_warehouse'].unique()

In [ ]:
# using Regular Expression for cleaning the data

pattern= '\d+' #matches one or more digits
masked = dirty_data['nearest_warehouse'].str.contains(pattern, regex= True)
result = dirty_data[masked]
result

In [ ]:
# replacing data
dirty_data['nearest_warehouse']= dirty_data['nearest_warehouse'].str.replace('\d+', '')
dirty_data.head()

In [ ]:
# corrected Data
dirty_data['nearest_warehouse'].unique()

In [ ]:
# Checking unique values in Order Total column

dirty_data['order_total'].unique()

In [ ]:
pattern= '[a-zA-Z\W]'
masked= dirty_data['order_total'].str.contains(pattern, regex= True)
result= dirty_data[masked]
result

In [ ]:
# Replacing anything from column containing ALphabet or special character.
# dirty_data['order_total']= dirty_data['order_total'].str.replace('[a-zA-Z\W]', '') # this method also removed the special characters like (.)
# correct code will be...

dirty_data['order_total']=  dirty_data['order_total'].str.replace('[^0-9.]', '')


In [ ]:
dirty_data['order_total']= dirty_data['order_total'].astype('float64')

In [ ]:
dirty_data.info()

In [ ]:
sum(dirty_data.order_total.values)

Checking For Name Description of Data










In [ ]:
dirty_data['nearest_warehouse'].unique()

In [ ]:
#Convert the values into similar names

dirty_data.nearest_warehouse= dirty_data.nearest_warehouse.str.title()

In [ ]:
dirty_data.nearest_warehouse.unique()

In [ ]:
#Count of Unique values in Column
warehouse= dirty_data.nearest_warehouse.value_counts()
print(warehouse)

**Bar Chart and Pie Chart**

In [ ]:
# Visualization of Unique  values in Nearest Warehouse Column

warehouse.plot(kind='bar', rot=1, color=['r','b', 'g'],figsize=(5,5)),
plt.title('Quantity of Warehouse\n', size= 15)
plt.ylabel('Counts');

In [ ]:
#plotting Pie chart for checking % share of each warehouse

plt.pie(warehouse,
        labels=['Thompson', 'Nickolson', 'Bakers'],
        autopct="%0.1f%%",
        explode=[0,.01,.1],
        startangle=60, radius=.8)
plt.title("Quantity of Warehouses", size= 15)
plt.show()

**Analyzing order price and Order Total**


1.   Order_Price= Price Before any discount or delivery charges
2.   Order_Total= Price after all discount and delivery charges applied








**Get Max and Min Values**

In [ ]:
# Checking for maximum valued order
dirty_data[dirty_data.order_price== dirty_data.order_price.max()]

In [ ]:
# Checking for Minimum valued order
dirty_data[dirty_data.order_price== dirty_data.order_price.min()]

Let's Check Calculation

In [ ]:
# Checking for the order total if all the discount and delivery charges applied is correct

dirty_data.loc[:,['order_price', 'delivery_charges', 'coupon_discount', 'order_total']].head(10)

In [ ]:
# Formula: (order_price - (order_price	*coupon_discount)/100) + delivery_charges= Final_Price

First Check for 10 recordes to correct and varify calculations

In [ ]:
for i, index in enumerate(range(0,10)):
    if(dirty_data['coupon_discount'][i]!= 0):
      dis_price= (dirty_data['order_price'][i]-((dirty_data['order_price'][i])* (dirty_data['coupon_discount'][i])) /100) #calculation of discounted price
      price= dis_price + dirty_data['delivery_charges'][i] #adding delivery charges
      final_price= np.round(price,2)

    else:
      final_price= dirty_data['order_price'][i]+ dirty_data['delivery_charges'][i] #adding oorder price and delivery charges
      final_price= np.round(final_price,2) # rounding off upto 2 decimal places

    print(f'Order total of record at index {index} is: {final_price} ')

**Correcting the Entire Column**

In [ ]:
lis= list()
for i, index in enumerate(range(0, len(dirty_data['order_total']))):
  if dirty_data['coupon_discount'][i] != 0:
    dis_price= dirty_data['order_price'][i] - ((dirty_data['order_price'][i]* (dirty_data['coupon_discount'][i])/100))
    price= dis_price + dirty_data['delivery_charges'][i]
    final_price= np.round(price, 2)
    lis.append(final_price)        #appending final prices to list
  else:
    price= dirty_data['order_price'][i] + dirty_data['delivery_charges'][i]
    final_price= np.round(price, 2)
    lis.append(final_price)

In [ ]:
print(f'Items (price) in the list : {len(lis)}')

**Outlier Check**

In [ ]:
dirty_data['order_total']= pd.Series(lis)

In [ ]:
Q1_order_total= dirty_data['order_total'].quantile(.25)       #Lower Quantile
Q3_order_total= dirty_data['order_total'].quantile(.75)        #Upper Quantile

print(f'Lower Quantile is : {Q1_order_total}')
print(f'Upper Quantile is : {Q3_order_total}')

In [ ]:
#Upper Quantile - Lower Quantile

IQR_order_total= Q3_order_total - Q1_order_total
print(f'Inter Quantile Range for Order Total is :{IQR_order_total}')

In [ ]:
lower_limit_order_total =  Q1_order_total -1.5*IQR_order_total
upper_limit_order_total = Q3_order_total +1.5*IQR_order_total

print(f'Lower Limit of Order Total is : {lower_limit_order_total}')
print(f'Upper Limit of Order Total is : {upper_limit_order_total}')

In [ ]:
#Maximum value of Order Total

dirty_data[dirty_data['order_total']== dirty_data['order_total'].max()]

In [ ]:
#Minimum Value of Order Total is

dirty_data[dirty_data['order_total']== dirty_data['order_total'].min()]

In [ ]:
outlier_dirty_data= dirty_data[(dirty_data['order_total']< lower_limit_order_total) | (dirty_data['order_total'] > upper_limit_order_total)]
outlier_dirty_data

In [ ]:
print(f'Number of Outliers in column : {outlier_dirty_data.shape[0]}')

**Removing Outliers**

In [ ]:
data_without_outlier= dirty_data[(dirty_data['order_total']> lower_limit_order_total) & (dirty_data['order_total']< upper_limit_order_total)]
print(f' Data Without any Outlier : {data_without_outlier.shape[0]}')

### **Finding Out Happy Customers**

In [ ]:
data_without_outlier['is_happy_customer'].unique()
satisfaction = data_without_outlier['is_happy_customer'].value_counts()
print(satisfaction)

In [ ]:
plt.figure(figsize=(15,5))

plt.subplot(121)
satisfaction.plot(kind= 'bar', color= ['g', 'b'], rot=  1)
plt.xticks(range(2), labels= ['Satisfy', 'Not Satisfy'])
plt.ylabel('Counts')
plt.title('Customer Satisfaction vs Non Satisfaction\n',  size = 15)

plt.subplot(122)
plt.pie(satisfaction, labels= ['Satisfy', 'Not Satisfy'], autopct= '%.1f%%', explode= (0,0.1),  shadow= True)
plt.title('Percentage of Satisfied Customer vs Non Satisfiied Customers')

plt.show